In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ESG 점수 데이터프레임 로드
keyword_1 = pd.read_csv("/content/drive/MyDrive/미래에셋/ESG점수/data/E기사_친환경.csv")
keyword_2 = pd.read_csv("/content/drive/MyDrive/미래에셋/ESG점수/data/E기사_탄소.csv")
keyword_3 = pd.read_csv("/content/drive/MyDrive/미래에셋/ESG점수/data/E기사_에너지.csv")
ESG_ratings = pd.read_csv("/content/drive/MyDrive/미래에셋/ESG점수/data/MSCI_ESG_Ratings.csv")

print("###")
print(keyword_1.head())

print("###")
print(keyword_2.head())

print("###")
print(keyword_3.head())

print("###")
print(ESG_ratings.head())


###
   index   Stock  Article_cnt  Negative_sim  Positive_sim       Score  Ranking
0     47  한국조선해양          816      0.013793      0.018112  819.524064      1.0
1     23      SK          816      0.011606      0.015263  818.984692      2.0
2      8     현대차          795      0.013590      0.020101  800.176552      3.0
3     27      LG          693      0.014880      0.026304  700.916262      4.0
4     86      GS          652      0.018298      0.031333  660.498660      5.0
###
   Stock  Article_cnt  Negative_sim  Positive_sim        Score  Ranking
0     SK         2804      0.007414      0.011284  2814.851249      1.0
1    현대차         2612      0.007683      0.011434  2621.796303      2.0
2   한국전력         2582      0.008142      0.012866  2594.196484      3.0
3     기아         2344      0.008712      0.013779  2355.875626      4.0
4  두산중공업         2339      0.010485      0.014938  2349.416017      5.0
###
     Stock  Article_cnt  Negative_sim  Positive_sim        Score  Ranking
0     LG

In [ ]:
stock = pd.read_csv("/content/drive/MyDrive/미래에셋/kospi/KOSPI_시가총액_상위500.csv")
stock = stock.loc[0:99]
ESG_SCORE = [] # 최종 점수


# 종목별 3가지 ESG 키워드 순위의 평균
for name in stock['종목명']:
  idx_1 = keyword_1.index[( keyword_1['Stock'] == name )].tolist()
  idx_2 = keyword_2.index[( keyword_2['Stock'] == name )].tolist()
  idx_3 = keyword_3.index[( keyword_3['Stock'] == name )].tolist()
  idx_4 = ESG_ratings.index[( ESG_ratings['stockName'] == name )].tolist()
  
  score = (keyword_1.at[idx_1[0], 'Ranking'] + keyword_2.at[idx_2[0], 'Ranking'] + keyword_3.at[idx_3[0], 'Ranking']) / 3 

  # MSCI 등급에 따른 추가 가중치
  score *= (1 + ESG_ratings.at[idx_4[0], 'esgRating'] / 10)

  ESG_SCORE.append([name, score])

final = pd.DataFrame(ESG_SCORE, columns = ['Stock', 'Score'])


# 점수 min-max 정규화
def minmax_norm(df):
    return (df - df.min()) / ( df.max() - df.min())

ESG_final = final['Score']
ESG_final = minmax_norm(ESG_final)
ESG_final_norm = pd.concat([final, ESG_final], axis = 1)
ESG_final_norm.columns = ['Stock', 'Score', 'Score_norm']
print(ESG_final_norm)


       Stock       Score  Score_norm
0       삼성전자   16.000000    0.072874
1     SK하이닉스   21.000000    0.110830
2      NAVER   98.000000    0.695344
3   삼성바이오로직스   43.200000    0.279352
4      삼성전자우  117.600000    0.844130
..       ...         ...         ...
95      신풍제약  105.600000    0.753036
96    현대차2우B  137.666667    0.996457
97     GS리테일   78.866667    0.550101
98   일진머티리얼즈   79.333333    0.553644
99     SK케미칼   68.600000    0.472166

[100 rows x 3 columns]


In [ ]:
ESG_final_norm.to_csv('/content/drive/MyDrive/ESG_final_norm.csv')